In [1]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

In [3]:
!mkdir ~/.kaggle

In [4]:
!cp "kaggle.json" ~/.kaggle/kaggle.json

In [5]:
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download rexhaif/emojifydata-en

100% 884M/886M [00:11<00:00, 78.9MB/s]
100% 886M/886M [00:11<00:00, 82.9MB/s]


In [7]:
!mkdir train
!unzip emojifydata-en.zip -d train

Archive:  emojifydata-en.zip
  inflating: train/dev.txt           
  inflating: train/emojitweets-01-04-2018.txt  
  inflating: train/test.txt          
  inflating: train/train.txt         


In [8]:
!ls -al train

total 2515116
drwxr-xr-x 2 root root       4096 Apr 28 14:57 .
drwxr-xr-x 1 root root       4096 Apr 28 14:57 ..
-rw-r--r-- 1 root root  209891747 Oct 24  2019 dev.txt
-rw-r--r-- 1 root root 1263662857 Oct 24  2019 emojitweets-01-04-2018.txt
-rw-r--r-- 1 root root  262392742 Oct 24  2019 test.txt
-rw-r--r-- 1 root root  839507033 Oct 24  2019 train.txt


In [9]:
import pandas as pd
import gc
import tqdm
import numpy as np
import re
import string
import tensorflow as tf
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, GRU, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Attention, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import SparseTopKCategoricalAccuracy

import tensorflow as tf
print("Tensorflow version " + tf.__version__)


Tensorflow version 2.12.0


In [10]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

Running on TPU  ['10.17.206.170:8470']


In [11]:
f = open('train/train.txt', 'r')
lines = f.readlines()
len(lines)

99514776

In [12]:
for i in tqdm.tqdm(range(len(lines))):
    lines[i] = lines[i].strip()
lines[:50]

100%|██████████| 99514776/99514776 [00:46<00:00, 2156775.12it/s]


['<START> O',
 'CeeC O',
 'is O',
 'going O',
 'to O',
 'be O',
 'another O',
 'Tboss O',
 'What O',
 'is O',
 '45 O',
 'million O',
 'Naira :face_with_tears_of_joy:',
 '<STOP> O',
 '',
 '<START> O',
 'This O',
 'gif O',
 'kills O',
 'me O',
 'Death O',
 'is O',
 'literally O',
 'gushing O',
 'towards O',
 'you O',
 'and O',
 'you O',
 'really O',
 'gon O',
 'do O',
 'a O',
 'whole O',
 '3point O',
 'turn :weary_face:',
 '<STOP> O',
 '',
 '<START> O',
 'LOVE O',
 'TEST O',
 'Raw O',
 'Real O',
 'JaDine :purple_heart:',
 '<STOP> O',
 '',
 '<START> O',
 'i O',
 'swear O',
 'we O',
 'dont O']

In [13]:
for i in tqdm.tqdm(range(len(lines))):
  if lines[i] != '' and lines[i][-1] == 'O':
    lines[i] = lines[i][:-2]
lines[:50]

100%|██████████| 99514776/99514776 [00:55<00:00, 1782885.11it/s]


['<START>',
 'CeeC',
 'is',
 'going',
 'to',
 'be',
 'another',
 'Tboss',
 'What',
 'is',
 '45',
 'million',
 'Naira :face_with_tears_of_joy:',
 '<STOP>',
 '',
 '<START>',
 'This',
 'gif',
 'kills',
 'me',
 'Death',
 'is',
 'literally',
 'gushing',
 'towards',
 'you',
 'and',
 'you',
 'really',
 'gon',
 'do',
 'a',
 'whole',
 '3point',
 'turn :weary_face:',
 '<STOP>',
 '',
 '<START>',
 'LOVE',
 'TEST',
 'Raw',
 'Real',
 'JaDine :purple_heart:',
 '<STOP>',
 '',
 '<START>',
 'i',
 'swear',
 'we',
 'dont']

In [14]:
cleaned_tweets = []
for i in tqdm.tqdm(range(len(lines))):
    if '<START>' in lines[i]:
        tweet = []
        if lines[i] != '<START>':
          tweet.append(lines[i].split(' ')[1])
    elif lines[i] == '<STOP>':
        cleaned_tweets.append(' '.join(tweet))
    else:
        if lines[i] != '':
            tweet.append(lines[i])
len(cleaned_tweets), cleaned_tweets[:50]

100%|██████████| 99514776/99514776 [01:00<00:00, 1652691.61it/s]


(6567625,
 ['CeeC is going to be another Tboss What is 45 million Naira :face_with_tears_of_joy:',
  'This gif kills me Death is literally gushing towards you and you really gon do a whole 3point turn :weary_face:',
  'LOVE TEST Raw Real JaDine :purple_heart:',
  'i swear we dont gotta look it finds us :face_with_tears_of_joy:',
  'We would like to wish everyone a very Happy New Year and all the best in 2018 :party_popper:',
  '15000 retweets a new song song off “ Swaecation “ part 2 of unforgettable go :speaking_head: 🏽 🏽',
  'just know ilysm k bye friend :purple_heart:',
  'Too glam to give a damn :sparkles:',
  ':clapping_hands: 🏼 :clapping_hands: 🏼 fuck that sicko',
  'Can I marry someone who understands all these memes :loudly_crying_face:',
  'My week on Twitter :party_popper: 2 Tweets See yours with',
  'rt this to gain seokjin stan mutuals follow everyone who retweets this and make sure to follow back :smiling_face_with_heart-eyes:',
  'theyre 14 so i mean :person_shrugging: 🏻 

In [15]:
del lines
gc.collect()

0

In [16]:
cleaned_tweets = [x for x in cleaned_tweets if x[0]!=':']

In [17]:
tweets = []
labels = []
for i in tqdm.tqdm(range(len(cleaned_tweets))):
  arr = cleaned_tweets[i].split(':')
  tweets.append(arr[0].strip())
  labels.append(arr[1])

100%|██████████| 6097411/6097411 [00:08<00:00, 709393.67it/s]


In [18]:
chosen_labels = list(pd.Series(labels).value_counts().iloc[:5].index)
chosen_labels = ['person_facepalming',
                'rolling_on_the_floor_laughing',
                'folded_hands',
                'weary_face',
                'thinking_face']
filtered_tweets = []
filtered_labels = []

for i in tqdm.tqdm(range(len(tweets))):
    if labels[i] in chosen_labels:
        filtered_tweets.append(tweets[i])
        filtered_labels.append(labels[i])
filtered_tweets[:10], filtered_labels[:10], len(filtered_tweets), len(filtered_labels)

100%|██████████| 6097411/6097411 [00:03<00:00, 1704576.84it/s]


(['This gif kills me Death is literally gushing towards you and you really gon do a whole 3point turn',
  'i have a whole closet full of clothes yet i never have anything to wear',
  'yooo the moon looks extra beautiful tonighttt',
  'Someone could be needing our services and that person could be on your timeline Kindly Retweet',
  'WANT 150FOLLOWERS FAST',
  'We are not crying You are CRYING',
  'Am i attracted to needy niccas',
  'i wanna just go back to sleep but i cant',
  'A girl cartwheeled into my DMs amp said You look like the type of man Id let take me for a meal',
  'Another acceptance letter'],
 ['weary_face',
  'person_facepalming',
  'weary_face',
  'folded_hands',
  'thinking_face',
  'weary_face',
  'thinking_face',
  'weary_face',
  'weary_face',
  'folded_hands'],
 661309,
 661309)

In [19]:
tweets = filtered_tweets
labels = filtered_labels
len(tweets)

661309

In [20]:
for i in tqdm.tqdm(range(len(tweets))):
    tweets[i] = tweets[i].lower()

100%|██████████| 661309/661309 [00:00<00:00, 1282809.74it/s]


In [21]:
tweets = [tweet.lower() for tweet in tweets]

tweets = [re.sub(r'#\w+', '[HASHTAG]', tweet) for tweet in tweets]
tweets = [re.sub(r'@\w+', '[MENTION]', tweet) for tweet in tweets]
tweets = [re.sub(r'http\S+', '[URL]', tweet) for tweet in tweets]

tweets = [re.sub(r'[^\w\s]', '', tweet) for tweet in tweets]

for i in tqdm.tqdm(range(len(tweets))):
  arr = tweets[i].split(' ')
  arr = [x for x in arr if x != '']
  tweets[i] = ' '.join(arr)

100%|██████████| 661309/661309 [00:01<00:00, 468728.75it/s]


In [22]:
label_dict = {}
for label in labels:
    if label not in label_dict:
        label_dict[label] = len(label_dict)

label_ids = [label_dict[label] for label in labels]
label_ids[:10]

[0, 1, 0, 2, 3, 0, 3, 0, 0, 2]

In [23]:
tokenized_tweets = [tweet.strip().split() for tweet in tweets]

In [24]:
max_words = 50000
max_length = 32

with tpu_strategy.scope():
  tokenizer = Tokenizer(num_words=max_words)
  tokenizer.fit_on_texts(tokenized_tweets)
  sequences = tokenizer.texts_to_sequences(tokenized_tweets)
  padded_tweets = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')


In [28]:
validation_split = 0.1

num_validation_samples = int(validation_split * len(padded_tweets))
x_train = padded_tweets[:-num_validation_samples]
y_train = np.array(label_ids[:-num_validation_samples])
x_val = padded_tweets[-num_validation_samples:]
y_val = np.array(label_ids[-num_validation_samples:])

In [26]:
batch_size = 1024
num_epochs = 30
lr = 1e-5

with tpu_strategy.scope():
  inputs = Input(shape=(32))
  embedding = Embedding(input_dim=4096, output_dim=1024, input_length=max_length)(inputs)

  out_layer = embedding
  for i in range(2):
    gru = GRU(1024, activation='relu', return_sequences=True)(out_layer)
    batch_norm = BatchNormalization()(gru)
    dropout = Dropout(0.5)(batch_norm)
    attention = Attention()([dropout, dropout])
    out_layer = Concatenate(axis=-1)([dropout, attention])

  gru = GRU(512, activation='relu')(out_layer)
  batch_norm = BatchNormalization()(gru)
  dropout = Dropout(0.5)(batch_norm)

  outputs = Dense(5, activation='softmax')(dropout)

  optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
  model = Model(inputs=inputs, outputs=outputs)

  model.compile(optimizer=optimizer, loss=loss_fn, metrics=[tf.keras.metrics.SparseCategoricalAccuracy()], steps_per_execution=32)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 32, 1024)     4194304     ['input_1[0][0]']                
                                                                                                  
 gru (GRU)                      (None, 32, 1024)     6297600     ['embedding[0][0]']              
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 1024)    4096        ['gru[0][0]']                    
 alization)                                                                                   

In [ ]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=num_epochs, validation_data=(x_val, y_val), 
          callbacks=[EarlyStopping(patience=5)])


In [31]:
inverse_label_dict = {}
for k,v in label_dict.items():
    inverse_label_dict[v] = k
inverse_label_dict

{0: 'weary_face',
 1: 'person_facepalming',
 2: 'folded_hands',
 3: 'thinking_face',
 4: 'rolling_on_the_floor_laughing'}

In [46]:
preds = []

new_tweet = tweets[-(num_validation_samples):]
new_tweet_tokens = tokenizer.texts_to_sequences(new_tweet)
new_tweet_padded = pad_sequences(new_tweet_tokens, maxlen=max_length, padding='post', truncating='post')

y_pred_probs = model.predict([new_tweet_padded])
y_pred = np.argmax(y_pred_probs, axis=1)
print(y_pred)

2067/2067 [==============================] - 38s 18ms/step
[2 0 0 ... 0 2 0]


In [47]:
df = pd.DataFrame()
df['tweets'] = new_tweet
df['y_true'] = y_val
df['y_true'] = df['y_true'].apply(lambda x: inverse_label_dict[x])
df['y_pred'] = y_pred
df['y_pred'] = df['y_pred'].apply(lambda x: inverse_label_dict[x])

df.to_csv("results.csv", index=None)
df

,tweets,y_true,y_pred
0,rt this and in 5 minutes good luck will come,folded_hands,folded_hands
1,ima miss this in august,weary_face,weary_face
2,im craving a steers burger i cant eat anything...,weary_face,weary_face
3,all too well sir hope your perfect skin kiehls...,folded_hands,folded_hands
4,whoever i get in a relationship with is gonna ...,weary_face,folded_hands
...,...,...,...
66125,i dont know how im going to stream series if m...,weary_face,thinking_face
66126,ive never learned to like it mugs or buckets,rolling_on_the_floor_laughing,thinking_face
66127,i wish i was with my valentine rn,weary_face,weary_face
66128,2 days of my cleanse downeternity to go,weary_face,folded_hands


In [53]:
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(y_val, y_pred), accuracy_score(y_val, y_pred) 

(array([[ 7941,  4101,  1226,  2361,   571],
        [  660,  7970,   450,  2024,   378],
        [  438,   766, 13520,  1374,   199],
        [  313,  1323,   495,  9838,   252],
        [  490,  2517,   719,  2469,  3735]]),
 0.6502948737335551)